# Configuration

In [114]:

import numpy as np
import matplotlib.pyplot as plt

from gp_ima.ima import C_ima_digamma, C_ima_sample
import GPy
from tueplots import bundles, figsizes

In [ ]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '.')

In [ ]:
from analysis import plot_typography, estimate2uniform, generate_moebius_data, format_violin, RED, BLUE, calc_mcc

In [ ]:
USETEX = True

In [ ]:
plt.rcParams.update(bundles.neurips2022(usetex=USETEX))
plt.rcParams.update({
    'text.latex.preamble': [r'\usepackage{amsfonts}', # mathbb
                            r'\usepackage{amsmath}'] # boldsymbol
})

In [ ]:
plot_typography(usetex=USETEX, small=12, medium=16, big=20)

# Functions

In [ ]:
def train_bayesian_gplvm(X, dim, num_samples_c_ima, num_restarts, num_seeds, seed):
    cimas_sparse = []
    cimas_sparse_prior = []
    zs_sparse = []
    zs_uni_sparse = []
    np.random.seed(seed)
    for i in range(num_seeds):
        kernel = GPy.kern.RBF(dim, ARD=False) + GPy.kern.Bias(dim)
        m = GPy.models.BayesianGPLVM(np.asarray(X), dim, kernel=kernel, num_inducing=20)
        m.likelihood = GPy.likelihoods.Gaussian(variance=1e-4)
        cimas_sparse_prior.append(np.mean([C_ima_sample(m) for _ in range(num_samples_c_ima)]))
        m.optimize_restarts(num_restarts, optimizer='lbfgs')

        cimas_sparse.append(np.mean([C_ima_sample(m) for _ in range(num_samples_c_ima)]))
        zs_sparse.append(m.X.mean)
        zs_uni_sparse.append(estimate2uniform(zs_sparse[-1]))

    return cimas_sparse, cimas_sparse_prior, zs_sparse , zs_uni_sparse

def train_gplvm(X, dim, num_samples_c_ima, num_restarts, num_seeds, seed):
    cimas = []
    cimas_prior = []
    zs = []
    zs_uni = []
    np.random.seed(seed)
    for i in range(num_seeds):
        kernel = GPy.kern.RBF(dim, ARD=False) + GPy.kern.Bias(dim)
        m = GPy.models.GPLVM(np.asarray(X), dim, kernel=kernel)
        m.likelihood = GPy.likelihoods.Gaussian(variance=1e-6)
        cimas_prior.append(np.mean([C_ima_sample(m) for _ in range(num_samples_c_ima)]))
        m.optimize_restarts(num_restarts, optimizer='lbfgs')

        cimas.append(np.mean([C_ima_sample(m) for _ in range(num_samples_c_ima)]))
        zs.append(m.X.values)
        zs_uni.append(estimate2uniform(zs[-1]))

    return cimas, cimas_prior, zs , zs_uni



# Train models

In [123]:
NUM_DATA = 500
SEED = 42
NUM_SEEDS = 5



## 2D

In [125]:

NUM_SAMPLES_C_IMA = 100
NUM_RESTARTS = 5
DIM = LATENT_DIM = OBS_DIM = 2

np.random.seed(SEED)
Z, X, c = generate_moebius_data(NUM_DATA, LATENT_DIM, OBS_DIM)

cimas_sparse_2d, cimas_sparse_prior_2d, zs_sparse_2d, zs_uni_sparse_2d = train_bayesian_gplvm(X, DIM, NUM_SAMPLES_C_IMA,
                                                                                              NUM_RESTARTS, NUM_SEEDS,
                                                                                              SEED)
mccs_sparse_2d = [calc_mcc(z, Z) for z in zs_uni_sparse_2d]

NUM_RESTARTS = 2
cimas_2d, cimas_prior_2d, zs_2d , zs_uni_2d = train_gplvm(X, DIM, NUM_SAMPLES_C_IMA, NUM_RESTARTS, NUM_SEEDS, SEED)
mccs_2d = [calc_mcc(z, Z) for z in zs_uni_2d]

Optimization restart 1/5, f = -2596.4202124419394
Optimization restart 2/5, f = -2596.4172090978896
Optimization restart 3/5, f = -2596.4198410951863
Optimization restart 4/5, f = -2596.4193691671594
Optimization restart 5/5, f = -2342.3127646550947
Optimization restart 1/5, f = -2596.4192268388188
Optimization restart 2/5, f = -2596.4194532763227
Optimization restart 3/5, f = -2353.80913556185
Optimization restart 4/5, f = -2169.0996624777918
Optimization restart 5/5, f = -2281.748069107265
Optimization restart 1/5, f = -2596.419515285671
Optimization restart 2/5, f = -2316.909745619596
Optimization restart 3/5, f = -2297.624583302467
Optimization restart 4/5, f = -2596.417724017836
Optimization restart 5/5, f = -2596.418611314886
Optimization restart 1/5, f = -2596.418841509837
Optimization restart 2/5, f = -2395.764986532248
Optimization restart 3/5, f = -2596.419911639028
Optimization restart 4/5, f = -2596.419147393385
Optimization restart 5/5, f = -2596.417172499977
Optimization 

In [126]:
cimas_2d

[0.0022708264143404175,
 0.0022708264143404175,
 0.0022708264143404175,
 0.0022708264143404175,
 0.0022708264143404175]

In [ ]:
LABELPAD = 1
TICK_PADDING = 2
IDX = 0
IDX_SPARSE = 1
fig = plt.figure(figsize=figsizes.neurips2022(nrows=1, ncols=2, rel_width=1)['figure.figsize'])

ax = fig.add_subplot(141)
ax.scatter(Z[:, 0], Z[:, 1], c=c, cmap="hsv", label="Latents")

ax2 = fig.add_subplot(142)
ax2.scatter(X[:, 0], X[:, 1], c=c, cmap="hsv", label="Observations")

ax3 = fig.add_subplot(143)
ax3.scatter(zs_uni_2d[IDX][:, 0], zs_uni_2d[IDX][:, 1], c=c, cmap="hsv", label="Rec. (GPLVM)")


ax4 = fig.add_subplot(144)
ax4.scatter(zs_uni_sparse_2d[IDX_SPARSE][:, 0], zs_uni_sparse_2d[IDX_SPARSE][:, 1], c=c, cmap="hsv", label="Rec. (Sparse GPLVM)")


# Remove ticks and labels and set which side to label
ticksoff = dict(labelleft=False, labelright=False, left=False, right=False, labelbottom=False, bottom=False)
ax.tick_params(axis="both", **ticksoff)
ax2.tick_params(axis="both", **ticksoff)
ax3.tick_params(axis="both", **ticksoff)
ax4.tick_params(axis="both", **ticksoff)

ax.set_title("Latents")
ax2.set_title("Observations")
ax3.set_title("Rec. (GPLVM)")
ax4.set_title("Rec. (Sparse GPLVM)")



plt.savefig("gplvm_ima.svg")

## 3D

In [102]:
NUM_SEEDS = 5
NUM_RESTARTS = 5
DIM = LATENT_DIM = OBS_DIM = 3

np.random.seed(SEED)
Z, X, c = generate_moebius_data(NUM_DATA, LATENT_DIM, OBS_DIM)

# cimas_sparse_3d, cimas_sparse_prior_3d, zs_sparse_3d, zs_uni_sparse_3d = train_bayesian_gplvm(X, DIM, NUM_SAMPLES_C_IMA,
#                                                                                               NUM_RESTARTS, NUM_SEEDS,
#                                                                                               SEED)
# mccs_sparse_3d = [calc_mcc(z, Z) for z in zs_uni_sparse_3d]

NUM_RESTARTS = 2
cimas_3d, cimas_prior_3d, zs_3d , zs_uni_3d = train_gplvm(X, DIM, NUM_SAMPLES_C_IMA, NUM_RESTARTS, NUM_SEEDS, SEED)
mccs_3d = [calc_mcc(z, Z) for z in zs_uni_3d]

Optimization restart 1/2, f = -8892.333048554805
Optimization restart 2/2, f = -7410.4127082568075
Optimization restart 1/2, f = -8892.333048554805
Optimization restart 2/2, f = -7437.7935463229505
Optimization restart 1/2, f = -8892.333048554805
Optimization restart 2/2, f = -4418.5988026243385
Optimization restart 1/2, f = -8892.333048554805
Optimization restart 2/2, f = -7377.2520684539395
Optimization restart 1/2, f = -8892.333048554805
Optimization restart 2/2, f = -7018.233468601604


## 5D

In [103]:
NUM_RESTARTS = 5
DIM = LATENT_DIM = OBS_DIM = 5

np.random.seed(SEED)
Z, X, c = generate_moebius_data(NUM_DATA, LATENT_DIM, OBS_DIM)

# cimas_sparse_5d, cimas_sparse_prior_5d, zs_sparse_5d, zs_uni_sparse_5d = train_bayesian_gplvm(X, DIM, NUM_SAMPLES_C_IMA,
#                                                                                               NUM_RESTARTS, NUM_SEEDS,
#                                                                                               SEED)
# mccs_sparse_5d = [calc_mcc(z, Z) for z in zs_uni_sparse_5d]

NUM_RESTARTS = 2
cimas_5d, cimas_prior_5d, zs_5d , zs_uni_5d = train_gplvm(X, DIM, NUM_SAMPLES_C_IMA, NUM_RESTARTS, NUM_SEEDS, SEED)
mccs_5d = [calc_mcc(z, Z) for z in zs_uni_5d]

Optimization restart 1/2, f = -14791.341945542168
Optimization restart 2/2, f = -13378.133889645942
Optimization restart 1/2, f = -14791.341945542168
Optimization restart 2/2, f = -13324.290807451509
Optimization restart 1/2, f = -14791.341945542168
Optimization restart 2/2, f = -13381.917296372525
Optimization restart 1/2, f = -14791.341945542168
Optimization restart 2/2, f = -13368.249408214022
Optimization restart 1/2, f = -14791.341945542168
Optimization restart 2/2, f = -8849.104455640243


## 8D

In [104]:
NUM_RESTARTS = 5
DIM = LATENT_DIM = OBS_DIM = 8

np.random.seed(SEED)
Z, X, c = generate_moebius_data(NUM_DATA, LATENT_DIM, OBS_DIM)

# cimas_sparse_8d, cimas_sparse_prior_8d, zs_sparse_8d, zs_uni_sparse_8d = train_bayesian_gplvm(X, DIM, NUM_SAMPLES_C_IMA,
#                                                                                               NUM_RESTARTS, NUM_SEEDS,
#                                                                                               SEED)
# mccs_sparse_8d = [calc_mcc(z, Z) for z in zs_uni_sparse_8d]

NUM_RESTARTS = 2
cimas_8d, cimas_prior_8d, zs_8d , zs_uni_8d = train_gplvm(X, DIM, NUM_SAMPLES_C_IMA, NUM_RESTARTS, NUM_SEEDS, SEED)
mccs_8d = [calc_mcc(z, Z) for z in zs_uni_8d]

Optimization restart 1/2, f = -23605.080441510898
Optimization restart 2/2, f = -22553.347284228137
Optimization restart 1/2, f = -23605.080441510898
Optimization restart 2/2, f = -16072.580273599538
Optimization restart 1/2, f = -23605.080441510898
Optimization restart 2/2, f = -22346.279311552014
Optimization restart 1/2, f = -23605.080441510898
Optimization restart 2/2, f = -16077.480762654586
Optimization restart 1/2, f = -23605.080441510898
Optimization restart 2/2, f = -22432.519254631665


## 10D

In [105]:
NUM_RESTARTS = 5
DIM = LATENT_DIM = OBS_DIM = 10

np.random.seed(SEED)
Z, X, c = generate_moebius_data(NUM_DATA, LATENT_DIM, OBS_DIM)

# cimas_sparse_10d, cimas_sparse_prior_10d, zs_sparse_10d, zs_uni_sparse_10d = train_bayesian_gplvm(X, DIM,
#                                                                                                   NUM_SAMPLES_C_IMA,
#                                                                                                   NUM_RESTARTS,
#                                                                                                   NUM_SEEDS, SEED)
# mccs_sparse_10d = [calc_mcc(z, Z) for z in zs_uni_sparse_10d]

NUM_RESTARTS = 2
cimas_10d, cimas_prior_10d, zs_10d , zs_uni_10d = train_gplvm(X, DIM, NUM_SAMPLES_C_IMA, NUM_RESTARTS, NUM_SEEDS, SEED)
mccs_sparse_10d = [calc_mcc(z, Z) for z in zs_uni_10d]

Optimization restart 1/2, f = -29465.050754955646
Optimization restart 2/2, f = -28091.94848409862
Optimization restart 1/2, f = -29465.050754955646
Optimization restart 2/2, f = -21267.76373936543
Optimization restart 1/2, f = -29465.050754955646
Optimization restart 2/2, f = -27948.54919100292
Optimization restart 1/2, f = -29465.050754955646
Optimization restart 2/2, f = -21267.763948091633
Optimization restart 1/2, f = -29465.050754955646
Optimization restart 2/2, f = -21267.763851184172


# Plots

In [ ]:
D = 100
C_ima_digamma(D, D)

In [ ]:
Ds = np.logspace(0, 3, 1000).astype(int)
plt.plot(Ds, [C_ima_digamma(D, D) for D in Ds])

In [ ]:
Ds = np.logspace(0, 3, 1000).astype(int)
plt.plot(Ds, [C_ima_digamma(max(1, int(np.log(D))), D) for D in Ds])

In [127]:
cimas = [cimas_2d, cimas_3d, cimas_5d, cimas_8d, cimas_10d]
cimas_prior = [cimas_prior_2d, cimas_prior_3d, cimas_prior_5d, cimas_prior_8d, cimas_prior_10d]

from os.path import isfile

if not isfile("cimas.npz"):
    np.savez("cimas.npz", cimas=cimas, cimas_prior=cimas_prior)
else:
    raise FileExistsError("File already exists")

In [131]:
# fig = plt.figure(figsize=figsizes.neurips2022(nrows=1, ncols=1, rel_width=1)['figure.figsize'])
#
# ax = fig.add_subplot(111)

vp2= plt.violinplot([np.log10(g) for g in cimas], showmedians=True)
format_violin(vp2, RED)
plt.show()

## Violin plots


In [117]:
LABELPAD = 2
TICK_PADDING = 2

fig = plt.figure(figsize=figsizes.neurips2022(nrows=1, ncols=2)['figure.figsize'])


"""MCC vs CIMA over different gamma"""
ax = fig.add_subplot(121)
ax.grid(True, which="both", ls="-.")


# Remove ticks and labels and set which side to label
ticksoff = dict(labelleft=False, labelright=False, left=False, right=False)
ax.tick_params(axis="y", **ticksoff)


# MCC
vp= ax.violinplot([np.log10(g) for g in cimas_prior[0]], showmedians=True)
format_violin(vp,BLUE)

# plt.legend(["a", "b", "c", "d", "e"])


In [ ]:
# to make the grid behind both plots: https://stackoverflow.com/a/55618417/16912032

LABELPAD = 2
TICK_PADDING = 2

fig = plt.figure(figsize=figsizes.neurips2022(nrows=1, ncols=2)['figure.figsize'])


"""MCC vs CIMA over different gamma"""
ax = fig.add_subplot(121)
ax.grid(True, which="both", ls="-.")


# Remove ticks and labels and set which side to label
ticksoff = dict(labelleft=False, labelright=False, left=False, right=False)
ax.tick_params(axis="y", **ticksoff)


# MCC
vp= ax.violinplot([np.log10(g) for g in cimas_prior], showmedians=True)
format_violin(vp,BLUE)
# CIMA
# ax2.set_axisbelow(True)
vp2= ax.violinplot([np.log10(g) for g in cimas_priord], showmedians=True)
format_violin(vp2, RED)


ax_cima.set_ylabel("$\log_{10}c_{\mathrm{IMA}}$", labelpad=LABELPAD)
ax.set_ylabel("$\mathrm{MCC}$", labelpad=LABELPAD+17)

ax.set_xlabel("$\log_{10}\gamma^2$", labelpad=LABELPAD)
ax.set_xticklabels([0] + sorted(np.log10(moebius_df["gamma_square"]).astype(int).unique()))



plt.legend([vp['bodies'][0], vp2['bodies'][0]], ["$\mathrm{MCC}$", "$\log_{10}c_{\mathrm{IMA}}$"], loc='center left')



plt.savefig("moebius_mcc_cima_left.svg")